**Análise de redes dos institutos de economia da Unicamp e da UFRJ: um retrato da economia heterodoxa brasileira**

Rotina para avaliação da disciplina de *Social Network Analysis* (HO-450 B)

Autores: 

    Anna Carolina Martins    RA: 226035
    Julia Alencar Omizzolo   RA: 207515
    Luan Lima Mugarte        RA: 156302

A base de dados foi obtida em: <https://apps.webofknowledge.com/SCIELO_AdvancedSearch_input.do?SID=8B7VFuO4vi1WmhQ8UXF&product=SCIELO&search_mode=AdvancedSearch>.

Mais especificamente, a busca de dados foi refinada em área de pesquisa (Business & Economics) e organização (primeiramente Unicamp e depois UFRJ), gerando um arquivo de dados para cada universidade. O formato utilizado para a exportação dos dados do site é texto sem formatação (WIN, UTF-8). 

# Importando pacotes e funções necessários

Descrição dos pacotes:
\begin{itemize}
\item unidecode: utilizado para retirar acentuação dos nomes dos pesquisadores, considerando que a acentuação gerava problemas para identificação de nomes de pesquisadores iguais.

\item re: identifica padrões de objetos de texto (*strings*)

\item textdistance: identifica a distância ou similaridade entre *strings*, permitindo a identificação de nomes de pesquisadores parecidos

\item graph-tool e funções associadas: pacote de criação e visualização de grafos e estatísticas relacionadas

\item pandas: pacote que agrega um conjunto de funções para tratar DataFrames

\item seaborn, matplotlib e funções associadas: pacotes de visualização de dados

\item numpy: pacote de expressões e funções matemáticas do Python
\end{itemize}


In [1]:
# Pacotes
import unidecode
import re
import textdistance
import graph_tool as gt
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

# Funções
from graph_tool.draw import graph_draw, graphviz_draw
from graph_tool import Graph, topology
%matplotlib inline

# Tratando os dados do IE - Unicamp

## Importando os dados

In [2]:
# Criando o caminho do arquivo de dados
path = 'Data/savedrecs.txt'

# É utilizada a função pd.read_csv, que importa os dados em um DataFrame.
# Ademais, além do caminho para o arquivo, dois argumentos adicionais são dados:
# sep, que indica que o arquivo é separado por tabulação (\t)
# encoding, que indica que o arquivo é codificado em utf-8
df = pd.read_csv(path, sep='\t', encoding='utf-8')

## Tratando os dados

In [3]:
# Verificando os dados no DataFrame
df

,PT,AU,BE,TI,X1,Y1,Z1,SO,LA,DT,...,EP,DI,EC,C2,SC,UT,OA,HC,HP,DA
J,"Braga Junior, Sergio; Martínez, Marta Pagán; C...",NaN,"Greenwashing effect, attitudes, and beliefs in...",NaN,NaN,NaN,RAUSP Management Journal,English,research-article,Marketing,...,10.1108/rausp-08-2018-0070,Business,SciELO Brazil,Business & Economics,SCIELO:S2531-04882019000200006,gold,NaN,NaN,NaN,NaN
J,"Rocha, Leonardo Andrade; Khan, Ahmad Saeed; Li...",NaN,"O ""efeito nefasto"" da corrupção no Brasil: ""qu...",The 'detrimental effect' of corruption in Braz...,NaN,NaN,Nova Economia,Portuguese,research-article,corruption; quantile regression; instrumental ...,...,10.1590/0103-6351/3695,Economics,SciELO Brazil,Business & Economics,SCIELO:S0103-63512019000100277,gold,NaN,NaN,NaN,NaN
J,"Prates, Daniela Magalhães",NaN,Apresentação,NaN,NaN,NaN,Economia e Sociedade,Portuguese,editorial,NaN,...,10.1590/1982-3533.2019v28n1art001,Economics; Sociology,SciELO Brazil,Business & Economics; Sociology,SCIELO:S0104-06182019000100001,gold,NaN,NaN,NaN,NaN
J,"Romeiro, Ademar Ribeiro",NaN,Evitar o colapso ambiental requer o fim do cap...,NaN,NaN,NaN,Economia e Sociedade,Portuguese,book-review,NaN,...,10.1590/1982-3533.2019v28n1art13,Economics; Sociology,SciELO Brazil,Business & Economics; Sociology,SCIELO:S0104-06182019000100014,gold,NaN,NaN,NaN,NaN
J,"Ramos Júnior, Flavio Paulino; Santos, Isabela ...",NaN,Capital structure of Brazilian public companie...,Estructura de capital de empresas brasileñas: ...,NaN,NaN,Contaduría y administración,English,research-article,Capital structure; Indebtedness; Corporate fin...,...,10.22201/fca.24488410e.2018.1152,"Business, Finance; Management",SciELO Mexico,Business & Economics,SCIELO:S0186-10422019000100002,gold,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
J,"Meirelles, Antonio J. A.; Lima, Gilberto Tadeu",NaN,"Regimes de endividamento, fragilidade financei...",NaN,NaN,NaN,Estudos Econômicos (São Paulo),Portuguese,research-article,indebtedness; financial fragility; productive ...,...,10.1590/S0101-41612003000300005,Economics,SciELO Brazil,Business & Economics,SCIELO:S0101-41612003000300005,gold,NaN,NaN,NaN,NaN
J,"Alexandre, Michel; Canuto, Otaviano; Silveira,...",NaN,Microfundamentos de falência de bancos atacadi...,NaN,NaN,NaN,Estudos Econômicos (São Paulo),Portuguese,research-article,microfoundations of banking failures; banking ...,...,10.1590/S0101-41612003000200002,Economics,SciELO Brazil,Business & Economics,SCIELO:S0101-41612003000200002,gold,NaN,NaN,NaN,NaN
J,"Lima, Gilberto Tadeu; Meirelles, Antonio J. A.",NaN,"Mark-up bancário, conflito distributivo e util...",NaN,NaN,NaN,Revista Brasileira de Economia,Portuguese,research-article,NaN,...,10.1590/S0034-71402003000100004,Economics,SciELO Brazil,Business & Economics,SCIELO:S0034-71402003000100004,gold,NaN,NaN,NaN,NaN
J,"Marçal, Emerson Fernandes; Pereira, Pedro Luiz...",NaN,Paridade do poder de compra: testando dados br...,NaN,NaN,NaN,Revista Brasileira de Economia,Portuguese,research-article,NaN,...,10.1590/S0034-71402003000100006,Economics,SciELO Brazil,Business & Economics,SCIELO:S0034-71402003000100006,gold,NaN,NaN,NaN,NaN


In [4]:
# Criando um index numérico crescente
df['ID'] = range(0, len(df))
df = df.set_index('ID')

In [5]:
# Retirando valores nulos (NaN)
df = df.dropna(axis=1, how='all')

In [6]:
# Verificando os dados para um artigo, a fim de obter as colunas relevantes
df.iloc[0]

PT    Braga Junior, Sergio; Martínez, Marta Pagán; C...
BE    Greenwashing effect, attitudes, and beliefs in...
TI                                                  NaN
X1                                                  NaN
Y1                                                  NaN
Z1                             RAUSP Management Journal
SO                                              English
LA                                     research-article
DT                                            Marketing
DE                                                  NaN
X5                                                  NaN
Y5                                                  NaN
Z5    Abstract Purpose The purpose of this paper is ...
AB                                                  NaN
X4                                                  NaN
Y4                                                  NaN
Z4    [Correa, Caroline Miranda] Universidade Estadu...
EM              , Sergio Silva Braga Junior/A-67

## Extraindo as colunas relevantes

In [7]:
# As colunas abaixo foram as colunas escolhidas como as relevantes para o trabalho

# Reajustando o DataFrame para conter somente as colunas escolhidas
df = df[['PT', 'BE', 'Z1', 'Z4', 'U2', 'PD']]

In [8]:
# Verificando o DataFrame
df

,PT,BE,Z1,Z4,U2,PD
ID,,,,,,
0,"Braga Junior, Sergio; Martínez, Marta Pagán; C...","Greenwashing effect, attitudes, and beliefs in...",RAUSP Management Journal,"[Correa, Caroline Miranda] Universidade Estadu...",Universidade de São Paulo,2019
1,"Rocha, Leonardo Andrade; Khan, Ahmad Saeed; Li...","O ""efeito nefasto"" da corrupção no Brasil: ""qu...",Nova Economia,"[Poz, Maria Ester Soares Dal] Universidade Est...",Nova Economia,2019
2,"Prates, Daniela Magalhães",Apresentação,Economia e Sociedade,"[Prates, Daniela Magalhães] revista Economia e...",Instituto de Economia da Universidade Estadual...,2019
3,"Romeiro, Ademar Ribeiro",Evitar o colapso ambiental requer o fim do cap...,Economia e Sociedade,"[Romeiro, Ademar Ribeiro] Universidade Estadua...",Instituto de Economia da Universidade Estadual...,2019
4,"Ramos Júnior, Flavio Paulino; Santos, Isabela ...",Capital structure of Brazilian public companie...,Contaduría y administración,"[Gaio, Luiz Eduardo] Universidade Estadual de ...","Facultad de Contaduría y Administración, UNAM",2019
...,...,...,...,...,...,...
227,"Meirelles, Antonio J. A.; Lima, Gilberto Tadeu","Regimes de endividamento, fragilidade financei...",Estudos Econômicos (São Paulo),"[Lima, Gilberto Tadeu] Universidade de São Pau...","Faculdade de Economia, Administração e Contabi...",2003
228,"Alexandre, Michel; Canuto, Otaviano; Silveira,...",Microfundamentos de falência de bancos atacadi...,Estudos Econômicos (São Paulo),"[Canuto, Otaviano] Universidade de São Paulo, ...","Faculdade de Economia, Administração e Contabi...",2003
229,"Lima, Gilberto Tadeu; Meirelles, Antonio J. A.","Mark-up bancário, conflito distributivo e util...",Revista Brasileira de Economia,"[Meirelles, Antonio J. A.] Universidade Estadu...",Fundação Getúlio Vargas,2003


In [9]:
# Nomeando as colunas
df.columns = [
    'autorcomacento', 'título', 'revista', 'universidade', 'localrevista',
    'ano'
]

## Criando listas e dicionários para a análise e construção dos grafos

### Lista de universidades

In [10]:
# Extraindo os nomes das universidades da respectiva coluna

listadeuniversidades = []
for i in range(len(df)):
    try:
        listadeuniversidades.append(
            df['universidade'][i].split('; ')[0].split('[')[1].split('] ')[1])
    except:
        listadeuniversidades.append(str(i))

In [11]:
# Agrupando somente o valores únicos na lista
listadeuniversidades = np.unique(listadeuniversidades)
listadeuniversidades = list(listadeuniversidades)

In [12]:
# Verificando a lista de universidades
# Como há vários valores númericos, optamos por não utilizar os dados de universidades para este trabalho.
listadeuniversidades

['109',
 '110',
 '120',
 '13',
 '130',
 '137',
 '166',
 '17',
 '173',
 '176',
 '178',
 '183',
 '201',
 '202',
 '23',
 '33',
 '39',
 '46',
 '59',
 '77',
 '88',
 'Banco Interamericano de Desenvolvimento, USA',
 'Centro Universitário FEI, Brazil',
 'Centro Universitário da Fundação Educacional Inaciana “Padre Sabóia de Medeiros”, Brazil',
 'Câmara Municipal de Diadema, Brasil',
 'Escola Superior de Propaganda e Marketing, Brazil',
 'Faculdades Alfa, Brasil',
 'Faculdades SPEI, Brasil',
 'Fundação Getúlio Vargas, Brazil',
 'Grupo de Estudos em Economia Industrial, Brazil',
 'Grupo de Pesquisa e Documentação sobre o Empresariamento da Saúde Henry Jouval Jr, Brasil',
 'IBMEC Business School',
 'Instituto Agronômico do Paraná, Brazil',
 'Instituto de Filosofia e Ciências Humanas',
 'Instituto de Pesquisa Econômica Aplicada, Brasil',
 'Instituto do Direito Púbico (IDP), Brazil',
 'Ministério da Ciência, Tecnologia e Inovação, Brazil',
 'Pontifícia Universidade Católica de Campinas, Brazil',
 '

### Dicionário para os *journals*

In [13]:
# Criando um dicionário para associar as revistas a um número específico (ID)
revistaid = {}

for i in range(len(df['revista'].unique())):
    k = df['revista'].unique()[i]
    v = i + 1
    revistaid[k] = v

revistaid

{'RAUSP Management Journal': 1,
 'Nova Economia': 2,
 'Economia e Sociedade': 3,
 'Contaduría y administración': 4,
 'Interações (Campo Grande)': 5,
 'Revista Contabilidade & Finanças': 6,
 'Brazilian Journal of Political Economy': 7,
 'BBR. Brazilian Business Review': 8,
 'Revista de Administração Contemporânea': 9,
 'Revista de Economia Contemporânea': 10,
 'Revista de Administração (São Paulo)': 11,
 'Revista de Gestão dos Países de Língua Portuguesa': 12,
 'América Latina en la historia económica': 13,
 'Revista de Administração de Empresas': 14,
 'Revista de Economia Política': 15,
 'Estudos Econômicos (São Paulo)': 16,
 'REAd. Revista Eletrônica de Administração (Porto Alegre)': 17,
 'RAM. Revista de Administração Mackenzie': 18,
 'BAR - Brazilian Administration Review': 19,
 'Revista de Administração Pública': 20,
 'Revista Brasileira de Economia': 21,
 'Pensamiento & Gestión': 22,
 'El trimestre económico': 23,
 'Revista Portuguesa e Brasileira de Gestão': 24,
 'Revista Brasile

In [14]:
# Criando a coluna 'revistaid' a partir do dicionário
for i in range(len(df)):
    for k, v in revistaid.items():
        if (df['revista'][i] == k):
            df.loc[i, 'revistaid'] = int(v)

In [15]:
# Verificando o DataFrame
df

,autorcomacento,título,revista,universidade,localrevista,ano,revistaid
ID,,,,,,,
0,"Braga Junior, Sergio; Martínez, Marta Pagán; C...","Greenwashing effect, attitudes, and beliefs in...",RAUSP Management Journal,"[Correa, Caroline Miranda] Universidade Estadu...",Universidade de São Paulo,2019,1.0
1,"Rocha, Leonardo Andrade; Khan, Ahmad Saeed; Li...","O ""efeito nefasto"" da corrupção no Brasil: ""qu...",Nova Economia,"[Poz, Maria Ester Soares Dal] Universidade Est...",Nova Economia,2019,2.0
2,"Prates, Daniela Magalhães",Apresentação,Economia e Sociedade,"[Prates, Daniela Magalhães] revista Economia e...",Instituto de Economia da Universidade Estadual...,2019,3.0
3,"Romeiro, Ademar Ribeiro",Evitar o colapso ambiental requer o fim do cap...,Economia e Sociedade,"[Romeiro, Ademar Ribeiro] Universidade Estadua...",Instituto de Economia da Universidade Estadual...,2019,3.0
4,"Ramos Júnior, Flavio Paulino; Santos, Isabela ...",Capital structure of Brazilian public companie...,Contaduría y administración,"[Gaio, Luiz Eduardo] Universidade Estadual de ...","Facultad de Contaduría y Administración, UNAM",2019,4.0
...,...,...,...,...,...,...,...
227,"Meirelles, Antonio J. A.; Lima, Gilberto Tadeu","Regimes de endividamento, fragilidade financei...",Estudos Econômicos (São Paulo),"[Lima, Gilberto Tadeu] Universidade de São Pau...","Faculdade de Economia, Administração e Contabi...",2003,16.0
228,"Alexandre, Michel; Canuto, Otaviano; Silveira,...",Microfundamentos de falência de bancos atacadi...,Estudos Econômicos (São Paulo),"[Canuto, Otaviano] Universidade de São Paulo, ...","Faculdade de Economia, Administração e Contabi...",2003,16.0
229,"Lima, Gilberto Tadeu; Meirelles, Antonio J. A.","Mark-up bancário, conflito distributivo e util...",Revista Brasileira de Economia,"[Meirelles, Antonio J. A.] Universidade Estadu...",Fundação Getúlio Vargas,2003,21.0


### DataFrame de artigos e respectivos autores e coautores

In [17]:
# Retirando acentuação dos nomes dos autores
df['autores'] = pd.Series([unidecode.unidecode(
    df['autorcomacento'][i]).title() for i in range(len(df))]).str.split('; ')



# Deletando a coluna que não será mais usada
#del df['autorcomacento']

In [18]:
df

,autorcomacento,título,revista,universidade,localrevista,ano,revistaid,autores
ID,,,,,,,,
0,"Braga Junior, Sergio; Martínez, Marta Pagán; C...","Greenwashing effect, attitudes, and beliefs in...",RAUSP Management Journal,"[Correa, Caroline Miranda] Universidade Estadu...",Universidade de São Paulo,2019,1.0,"[Braga Junior, Sergio, Martinez, Marta Pagan, ..."
1,"Rocha, Leonardo Andrade; Khan, Ahmad Saeed; Li...","O ""efeito nefasto"" da corrupção no Brasil: ""qu...",Nova Economia,"[Poz, Maria Ester Soares Dal] Universidade Est...",Nova Economia,2019,2.0,"[Rocha, Leonardo Andrade, Khan, Ahmad Saeed, L..."
2,"Prates, Daniela Magalhães",Apresentação,Economia e Sociedade,"[Prates, Daniela Magalhães] revista Economia e...",Instituto de Economia da Universidade Estadual...,2019,3.0,"[Prates, Daniela Magalhaes]"
3,"Romeiro, Ademar Ribeiro",Evitar o colapso ambiental requer o fim do cap...,Economia e Sociedade,"[Romeiro, Ademar Ribeiro] Universidade Estadua...",Instituto de Economia da Universidade Estadual...,2019,3.0,"[Romeiro, Ademar Ribeiro]"
4,"Ramos Júnior, Flavio Paulino; Santos, Isabela ...",Capital structure of Brazilian public companie...,Contaduría y administración,"[Gaio, Luiz Eduardo] Universidade Estadual de ...","Facultad de Contaduría y Administración, UNAM",2019,4.0,"[Ramos Junior, Flavio Paulino, Santos, Isabela..."
...,...,...,...,...,...,...,...,...
227,"Meirelles, Antonio J. A.; Lima, Gilberto Tadeu","Regimes de endividamento, fragilidade financei...",Estudos Econômicos (São Paulo),"[Lima, Gilberto Tadeu] Universidade de São Pau...","Faculdade de Economia, Administração e Contabi...",2003,16.0,"[Meirelles, Antonio J. A., Lima, Gilberto Tadeu]"
228,"Alexandre, Michel; Canuto, Otaviano; Silveira,...",Microfundamentos de falência de bancos atacadi...,Estudos Econômicos (São Paulo),"[Canuto, Otaviano] Universidade de São Paulo, ...","Faculdade de Economia, Administração e Contabi...",2003,16.0,"[Alexandre, Michel, Canuto, Otaviano, Silveira..."
229,"Lima, Gilberto Tadeu; Meirelles, Antonio J. A.","Mark-up bancário, conflito distributivo e util...",Revista Brasileira de Economia,"[Meirelles, Antonio J. A.] Universidade Estadu...",Fundação Getúlio Vargas,2003,21.0,"[Lima, Gilberto Tadeu, Meirelles, Antonio J. A.]"


In [19]:
# Identificando o número máximo de coautores

n = max([len(df['autores'][i]) for i in range(len(df))]) - 1

In [25]:
lista = ['autor'] + ['coautor'+str(i) for i in range(1,n+1)]


authors = {}
for i in range(n+1):
    authors[lista[i]] = []    

    for t, a in df['autores'].items():
                
        try:
            authors[lista[i]].append(a[i])
        except IndexError:
            authors[lista[i]].append('')
            
for k,v in authors.items():
    df[k] = v

In [26]:
# Verificando o DataFrame
df

,autorcomacento,título,revista,universidade,localrevista,ano,revistaid,autores,autor,coautor1,coautor2,coautor3,coautor4,coautor5,coautor6,coautor7
ID,,,,,,,,,,,,,,,,
0,"Braga Junior, Sergio; Martínez, Marta Pagán; C...","Greenwashing effect, attitudes, and beliefs in...",RAUSP Management Journal,"[Correa, Caroline Miranda] Universidade Estadu...",Universidade de São Paulo,2019,1.0,"[Braga Junior, Sergio, Martinez, Marta Pagan, ...","Braga Junior, Sergio","Martinez, Marta Pagan","Correa, Caroline Miranda","Moura-Leite, Rosamaria Cox","Da Silva, Dirceu",,,
1,"Rocha, Leonardo Andrade; Khan, Ahmad Saeed; Li...","O ""efeito nefasto"" da corrupção no Brasil: ""qu...",Nova Economia,"[Poz, Maria Ester Soares Dal] Universidade Est...",Nova Economia,2019,2.0,"[Rocha, Leonardo Andrade, Khan, Ahmad Saeed, L...","Rocha, Leonardo Andrade","Khan, Ahmad Saeed","Lima, Patricia Veronica Pinheiro Sales","Poz, Maria Ester Soares Dal","Oliveira, Fernando Porfirio Soares De",,,
2,"Prates, Daniela Magalhães",Apresentação,Economia e Sociedade,"[Prates, Daniela Magalhães] revista Economia e...",Instituto de Economia da Universidade Estadual...,2019,3.0,"[Prates, Daniela Magalhaes]","Prates, Daniela Magalhaes",,,,,,,
3,"Romeiro, Ademar Ribeiro",Evitar o colapso ambiental requer o fim do cap...,Economia e Sociedade,"[Romeiro, Ademar Ribeiro] Universidade Estadua...",Instituto de Economia da Universidade Estadual...,2019,3.0,"[Romeiro, Ademar Ribeiro]","Romeiro, Ademar Ribeiro",,,,,,,
4,"Ramos Júnior, Flavio Paulino; Santos, Isabela ...",Capital structure of Brazilian public companie...,Contaduría y administración,"[Gaio, Luiz Eduardo] Universidade Estadual de ...","Facultad de Contaduría y Administración, UNAM",2019,4.0,"[Ramos Junior, Flavio Paulino, Santos, Isabela...","Ramos Junior, Flavio Paulino","Santos, Isabela Dos","Gaio, Luiz Eduardo","Stefanelli, Nelson Oliveira","Passos, Ivan Carlin",,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,"Meirelles, Antonio J. A.; Lima, Gilberto Tadeu","Regimes de endividamento, fragilidade financei...",Estudos Econômicos (São Paulo),"[Lima, Gilberto Tadeu] Universidade de São Pau...","Faculdade de Economia, Administração e Contabi...",2003,16.0,"[Meirelles, Antonio J. A., Lima, Gilberto Tadeu]","Meirelles, Antonio J. A.","Lima, Gilberto Tadeu",,,,,,
228,"Alexandre, Michel; Canuto, Otaviano; Silveira,...",Microfundamentos de falência de bancos atacadi...,Estudos Econômicos (São Paulo),"[Canuto, Otaviano] Universidade de São Paulo, ...","Faculdade de Economia, Administração e Contabi...",2003,16.0,"[Alexandre, Michel, Canuto, Otaviano, Silveira...","Alexandre, Michel","Canuto, Otaviano","Silveira, Jose Maria Da",,,,,
229,"Lima, Gilberto Tadeu; Meirelles, Antonio J. A.","Mark-up bancário, conflito distributivo e util...",Revista Brasileira de Economia,"[Meirelles, Antonio J. A.] Universidade Estadu...",Fundação Getúlio Vargas,2003,21.0,"[Lima, Gilberto Tadeu, Meirelles, Antonio J. A.]","Lima, Gilberto Tadeu","Meirelles, Antonio J. A.",,,,,,


In [33]:
# Verificando os diferentes valores da coluna 'coautor7'
np.unique(authors.values())

array([dict_values([['Braga Junior, Sergio', 'Rocha, Leonardo Andrade', 'Prates, Daniela Magalhaes', 'Romeiro, Ademar Ribeiro', 'Ramos Junior, Flavio Paulino', 'Mello, Guilherme Santos', 'Pozzer, Carlos Eduardo', 'Batista, Alexandre Ricardo De Aragao', 'Maia, Alexandre Gori', 'Blikstad, Nicholas Magnus Deleuse', 'Telles, Tiago Santos', 'Marcato, Marilia Bassetti', 'Correa, Ana Carolina Costa', 'Campos, Matheus Leite', 'Katovich, Erik S.', 'Saes, Beatriz Macchione', 'Ferreira, Adriana Nunes', 'Santos, Claudio Hamilton Matos Dos', 'Lima, Fabiano Guasti', 'Moraes, Gustavo Herminio Salati Marcondes De', 'Rocha, Leonardo Andrade', 'Ruppert, Lidia', 'Hiratuka, Celio', 'Braga, Jose Carlos', 'Nascimento, Paulo Francisco Do', 'Arantes, Flavio', 'Sestelo, Jose Antonio De Freitas', 'Reydon, Bastiaan Philip', 'Conti, Bruno De', 'Chiarini, Tulio', 'Silveira, Rodrigo Lanna Franco Da', 'Maciel, Leandro Dos Santos', 'Madeira, Gilberto Dos Santos', 'Campos, Filipe Medeia De', 'Bastos, Pedro Paulo Zahlu

In [43]:
# Criando uma lista com todos os nomes dos autores e coautores
# É utilizado o slice[1:] para retirar os casos de espaço vazio ('')
autores = np.unique([*authors.values()])[1:]

In [44]:
# Verificando a lista de autores e coautores
autores

array(['Abib, Gustavo', 'Afonso, Jose Roberto',
       'Aggio, Gustavo De Oliveira', 'Albuquerque, Eduardo Da Motta E',
       'Albuquerque, Marconi Edson Esmeraldo', 'Alcadipani, Rafael',
       'Alcantara, Daniela P. R. De', 'Alcantara, Wenersamy Ramos De',
       'Alderete, Veronica', 'Alexandre, Michel',
       'Almeida, Carlos Alano Soares De', 'Alves, Andre Cherubini',
       'Alves, Elia Elisa Cia', 'Amitrano, Claudio Roberto',
       'Andrade, Daniel Caixeta', 'Andrade, Rogerio P. De',
       'Andrade, Rogerio Pereira De', 'Andrietta, Lucas Salvador',
       'Anunciacao, Pedro', 'Arantes, Flavio',
       'Arruda, Jose Jobson De Andrade', 'Assaf Neto, Alexandre',
       'Assis, Renato Linhares De', 'Augusto De Souza, Paulo Ramalho',
       'Bacic, Miguel', 'Baioco, Gisele Busichia', 'Balestrin, Alsones',
       'Ballini, Rosangela', 'Balsadi, Otavio Valentim',
       'Baltar, Paulo Eduardo', 'Barbieux, Denise',
       'Barone, Ricardo Strazzacappa', 'Barreto, Joao Carlos',
     

In [46]:
# Criando uma lista para os nomes dos autores separada em sobrenome e nomes.
autoressep = [autores[i].split(' ') for i in range(len(autores))]

# Verificando a lista
autoressep

[['Abib,', 'Gustavo'],
 ['Afonso,', 'Jose', 'Roberto'],
 ['Aggio,', 'Gustavo', 'De', 'Oliveira'],
 ['Albuquerque,', 'Eduardo', 'Da', 'Motta', 'E'],
 ['Albuquerque,', 'Marconi', 'Edson', 'Esmeraldo'],
 ['Alcadipani,', 'Rafael'],
 ['Alcantara,', 'Daniela', 'P.', 'R.', 'De'],
 ['Alcantara,', 'Wenersamy', 'Ramos', 'De'],
 ['Alderete,', 'Veronica'],
 ['Alexandre,', 'Michel'],
 ['Almeida,', 'Carlos', 'Alano', 'Soares', 'De'],
 ['Alves,', 'Andre', 'Cherubini'],
 ['Alves,', 'Elia', 'Elisa', 'Cia'],
 ['Amitrano,', 'Claudio', 'Roberto'],
 ['Andrade,', 'Daniel', 'Caixeta'],
 ['Andrade,', 'Rogerio', 'P.', 'De'],
 ['Andrade,', 'Rogerio', 'Pereira', 'De'],
 ['Andrietta,', 'Lucas', 'Salvador'],
 ['Anunciacao,', 'Pedro'],
 ['Arantes,', 'Flavio'],
 ['Arruda,', 'Jose', 'Jobson', 'De', 'Andrade'],
 ['Assaf', 'Neto,', 'Alexandre'],
 ['Assis,', 'Renato', 'Linhares', 'De'],
 ['Augusto', 'De', 'Souza,', 'Paulo', 'Ramalho'],
 ['Bacic,', 'Miguel'],
 ['Baioco,', 'Gisele', 'Busichia'],
 ['Balestrin,', 'Alsones']

In [47]:
# Criando um dicionário para associar o autor/coautor a um número (ID)
# Criando dicionário com os nomes dos autores
autoresid = {}
# Criando a primeira entrada do dicionário
k = autores[0]
autoresid[k] = 1
for i in range(1, len(autores)):
    k = autores[i]
    autoresid[k] = i + 1
    # Condicional para ajustar o ID de nomes semelhantes, porém diferentes
    if textdistance.damerau_levenshtein.normalized_similarity(
            autoressep[i - 1][0], autoressep[i][0]) > 0.7:
        if textdistance.damerau_levenshtein.normalized_similarity(
                autoressep[i - 1][1], autoressep[i][1]) > 0.7:
            
            autoresid[k] = i
            
    else:
        continue

In [48]:
# Verificando o dicionário
autoresid

{'Abib, Gustavo': 1,
 'Afonso, Jose Roberto': 2,
 'Aggio, Gustavo De Oliveira': 3,
 'Albuquerque, Eduardo Da Motta E': 4,
 'Albuquerque, Marconi Edson Esmeraldo': 5,
 'Alcadipani, Rafael': 6,
 'Alcantara, Daniela P. R. De': 7,
 'Alcantara, Wenersamy Ramos De': 8,
 'Alderete, Veronica': 9,
 'Alexandre, Michel': 10,
 'Almeida, Carlos Alano Soares De': 11,
 'Alves, Andre Cherubini': 12,
 'Alves, Elia Elisa Cia': 13,
 'Amitrano, Claudio Roberto': 14,
 'Andrade, Daniel Caixeta': 15,
 'Andrade, Rogerio P. De': 16,
 'Andrade, Rogerio Pereira De': 16,
 'Andrietta, Lucas Salvador': 18,
 'Anunciacao, Pedro': 19,
 'Arantes, Flavio': 20,
 'Arruda, Jose Jobson De Andrade': 21,
 'Assaf Neto, Alexandre': 22,
 'Assis, Renato Linhares De': 23,
 'Augusto De Souza, Paulo Ramalho': 24,
 'Bacic, Miguel': 25,
 'Baioco, Gisele Busichia': 26,
 'Balestrin, Alsones': 27,
 'Ballini, Rosangela': 28,
 'Balsadi, Otavio Valentim': 29,
 'Baltar, Paulo Eduardo': 30,
 'Barbieux, Denise': 31,
 'Barone, Ricardo Strazzaca

In [ ]:
# Arrumando casos manualmente
# Por exemplo, nomes ou sobrenomes escritos de forma errada ou abreviada
# Por exemplo, autores com nomes e sobrenomes iguais, porém autores diferentes
autoresid['Biasoto Junior, Geraldo'] = 45
autoresid['Coutinho, Mauricio Chalflin'] = 88
autoresid['Deos, Simone Silva De'] = 101
autoresid['Maia, Alexandre Gori'] = 179
autoresid['Moraes, Gustavo Inacio De'] = 202  # Falso Positivo
autoresid['Salles Filho, Sergio Luiz Monteiro'] = 272
autoresid['Santos Filho, Otaviano Canuto Dos'] = 63
autoresid['Silva, Ana Lucia Goncalves Da'] = 288  # Falso Positivo
autoresid['Silveira, Jose Maria Ferreira Jardim Da'] = 296

In [ ]:
# Verificando o dicionário
autoresid

In [ ]:
# Criando a coluna de autor com o ID
df['autor'] = df['autor'].map(autoresid)

# Criando as colunas de coautores com ID
for t in range(1, max([len(df['autores'].str.split('; ')[i]) for i in range(len(df))])):
    string = 'coautor' + str(t)
    df[string] = df[string].map(autoresid).fillna('')

In [ ]:
# Verificando o DataFrame
df

### Lista de número de artigos por autor/coautor

In [ ]:
# Lista de todos os ID's de autores
list(np.unique(list(autoresid.values())))

In [ ]:
# Dimensão dessa lista
len(list(np.unique(list(autoresid.values()))))

In [ ]:
# Criando um dicionário que associa o número de artigos publicados por autor ao seu ID
nartigos = {}
for i in list(np.unique(list(autoresid.values()))):
    k = i
    v = df.loc[df['autor'] == i].autor.count()
    for t in range(1, 8):
        string = 'coautor' + str(t)
        v = v + df.loc[df[string] == i].autor.count()
        nartigos[k] = v

In [ ]:
# Verificando o dicionário
nartigos

### Lista de edges

In [ ]:
# Criando uma lista dos edges a serem desenhados, associando o ID do autor com os respectivos ID's de seus coautores

# Cabe ressaltar que os autores que não possuem coautores (artigos de 1 só autor) são ignorados aqui.
# Eles serão adicionados mais tarde.

edges = []
for t in range(
    1,
        (max([len(df['autores'].str.split('; ')[i]) for i in range(len(df))]))):
    for i in df[df['coautor' + str(t)] != ''][['autor', 'coautor' + str(t)
                                               ]].itertuples(index=False,
                                                             name=None):
        edges.append(i)

In [ ]:
# Verificando as edges
edges

### DataFrame de referência

In [ ]:
# Criando uma lista de referência com o ID de todos os autores
referencia = [i[t] for i in edges for t in range(2)] + list(set(np.unique(list(
    autoresid.values()))) - set(np.unique([i[t] for i in edges for t in range(2)])))

In [ ]:
# Criando um DataFrame de referência para identificar os ID's dos autores no grafo a ser criado adiante.
referencia = pd.DataFrame(
    referencia, columns=['ID']).drop_duplicates().reset_index(drop=True)

In [ ]:
# Adicionando a coluna do nome dos autores e seus respectivos número de artigos publicados na base de dados
autoresid = {y: x for x, y in autoresid.items()}
referencia['autores'] = referencia['ID'].map(autoresid)
referencia['nartigos'] = referencia['ID'].map(nartigos)

In [ ]:
# Criando a coluna com os ID's
referencia['ID'] = referencia.index

In [ ]:
# Verificando o DataFrame de referência
referencia

# Tratando os dados do IE - UFRJ

Vale destacar que as etapas de tratamento de dados para da UFRJ são iguais ou muito parecidas com as utilizadas para a base de dados da Unicamp.

## Importando os dados

In [ ]:
# Criando o caminho do arquivo de dados
path = 'Data/savedrecsrj.txt'

# É utilizada a função pd.read_csv, que importa os dados em um DataFrame.
# Ademais, além do caminho para o arquivo, dois argumentos adicionais são dados:
# sep, que indica que o arquivo é separado por tabulação (\t)
# encoding, que indica que o arquivo é codificado em utf-8
dfrj = pd.read_csv(path, sep='\t', encoding='utf-8')

## Tratando os dados

In [ ]:
# Verificando os dados no DataFrame
dfrj

In [ ]:
# Criando um index numérico crescente
dfrj['ID'] = range(0, len(dfrj))
dfrj = dfrj.set_index('ID')

In [ ]:
# Retirando valores nulos (NaN)
dfrj = dfrj.dropna(axis=1, how='all')

## Extraindo as colunas relevantes

In [ ]:
# Extraindo as colunas relevantes
dfrj = dfrj[['PT', 'BE', 'Z1', 'Z4', 'U2', 'PD']]

In [ ]:
# Nomeando as colunas
dfrj.columns = [
    'autorcomacento', 'título', 'revista', 'universidade', 'localrevista',
    'ano'
]

## Criando listas e dicionários para a análise

### Lista de universidades

In [ ]:
# Extraindo os nomes das universidades da respectiva coluna

listadeuniversidades = []
for i in range(len(df)):
    try:
        listadeuniversidades.append(dfrj['universidade'][i].split('; ')
                                    [0].split('[')[1].split('] ')[1])
    except:
        listadeuniversidades.append(str(i))

# Agrupando somente o valores únicos na lista
listadeuniversidades = np.unique(listadeuniversidades)
listadeuniversidades = list(listadeuniversidades)

# Verificando a lista de universidades
listadeuniversidades

### Dicionário para os *journals*

In [ ]:
# Criando um dicionário para associar as revistas a um número específico (ID)
revistaidrj = {}

for i in range(len(dfrj['revista'].unique())):
    k = dfrj['revista'].unique()[i]
    v = i + 1
    revistaidrj[k] = v

In [ ]:
# Criando a coluna 'revistaid' a partir do dicionário
for i in range(len(dfrj)):
    for k, v in revistaidrj.items():
        if (dfrj['revista'][i] == k):
            dfrj.loc[i, 'revistaid'] = int(v)

In [ ]:
revistaidrj

### DataFrame de artigos e respectivos autores e coautores

In [ ]:
# Retirando acentuação dos nomes dos autores
dfrj['autores'] = [unidecode.unidecode(
    dfrj['autorcomacento'][i]).title() for i in range(len(dfrj))]

del dfrj['autorcomacento']

In [ ]:
# Identificando o número máximo de coautores

max([len(dfrj['autores'].str.split('; ')[i]) for i in range(len(dfrj))]) - 1

In [ ]:
# Novamente, verifica-se que a coluna de autores contém todos os nomes dos autores em um mesmo item do DataFrame. /
# É necessário separá-los.

# Criando listas de autor e respectivos coautores, com somente um item para cada nome de pesquisador.

# A lista de autor é facilmente criada.

autor = [dfrj['autores'][i].split('; ')[0] for i in range(len(dfrj))]

# Criar a lista de coautores é problemático. Como o número de coautores varia e como usamos a função /
# split e um indexador, essa combinação pode incorrer em IndexError ao utilizar um for loop. Logo,   /
# optamos por utilizar um for loop com a função try except para contornar eventuais IndexErrors.

coautor1 = []
coautor2 = []
coautor3 = []
coautor4 = []
coautor5 = []
coautor6 = []
coautor7 = []
coautor8 = []

# Adicionando os coautores às listas
# É utilizado try no for loop porque o index da lista dá erro caso não exista
for i in range(len(dfrj)):
    # Primeira lista de coautores
    try:
        coautor1.append(dfrj['autores'][i].split('; ')[1])
    except IndexError:
        coautor1.append('')

for i in range(len(dfrj)):
    # Segunda lista de coautores
    try:
        coautor2.append(dfrj['autores'][i].split('; ')[2])
    except IndexError:
        coautor2.append('')

for i in range(len(dfrj)):
    # Terceira lista de coautores
    try:
        coautor3.append(dfrj['autores'][i].split('; ')[3])
    except IndexError:
        coautor3.append('')

for i in range(len(dfrj)):
    # Quarta lista de coautores
    try:
        coautor4.append(dfrj['autores'][i].split('; ')[4])
    except IndexError:
        coautor4.append('')

for i in range(len(dfrj)):
    # Quinta lista de coautores
    try:
        coautor5.append(dfrj['autores'][i].split('; ')[5])
    except IndexError:
        coautor5.append('')

for i in range(len(dfrj)):
    # Sexta lista de coautores
    try:
        coautor6.append(dfrj['autores'][i].split('; ')[6])
    except IndexError:
        coautor6.append('')

for i in range(len(dfrj)):
    # Sétima lista de coautores
    try:
        coautor7.append(dfrj['autores'][i].split('; ')[7])
    except IndexError:
        coautor7.append('')

for i in range(len(dfrj)):
    # Oitava lista de coautores
    try:
        coautor8.append(dfrj['autores'][i].split('; ')[8])
    except IndexError:
        coautor8.append('')

# Adicionando as listas às colunas respectivas dos coautores
dfrj['autor'] = autor
dfrj['coautor1'] = coautor1
dfrj['coautor2'] = coautor2
dfrj['coautor3'] = coautor3
dfrj['coautor4'] = coautor4
dfrj['coautor5'] = coautor5
dfrj['coautor6'] = coautor6
dfrj['coautor7'] = coautor7
dfrj['coautor8'] = coautor8

In [ ]:
# Verificando o DataFrame
dfrj

In [ ]:
# Verificando os diferentes valores da coluna 'coautor7'
dfrj['coautor8'].unique()

In [ ]:
# Criando uma lista com todos os nomes dos autoresrj e coautoresrj
autoresrj = autor + coautor1 + coautor2 + coautor3 + \
    coautor4 + coautor5 + coautor6 + coautor7 + coautor8

In [ ]:
# Código para encontrar os valores únicos de uma lista
autoresrj = np.unique(autoresrj)
autoresrj = list(autoresrj)

In [ ]:
# Retirando o caso de quando não há coautor, ou seja, um espaço vazio ('')
autoresrj = autoresrj[1:]

In [ ]:
# Verificando a lista de autores e coautores
autoresrj

In [ ]:
# Criando uma lista para os nomes dos autores separada em sobrenome e nomes.
autoresrjsep = [autoresrj[i].split(' ') for i in range(len(autoresrj))]

# Verificando a lista
autoresrjsep

In [ ]:
# Criando um dicionário para associar o autor/coautor a um número (ID)
autoresrjid = {}
# Criando a primeira entrada do dicionário
k = autoresrj[0]
autoresrjid[k] = 1

for i in range(1, len(autoresrj)):
    k = autoresrj[i]
    autoresrjid[k] = i + 1
    # Condicional para ajustar o ID de nomes semelhantes, porém diferentes
    if textdistance.damerau_levenshtein.normalized_similarity(
            autoresrjsep[i - 1][0], autoresrjsep[i][0]) > 0.7:
        if textdistance.damerau_levenshtein.normalized_similarity(
                autoresrjsep[i - 1][1], autoresrjsep[i][1]) > 0.7:
            autoresrjid[k] = i
    else:
        continue

In [ ]:
autoresrjid

In [ ]:
# Arrumando casos manualmente
autoresrjid['Batista, Jorge Chami'] = 66
autoresrjid['Cardoso, Fernanda Graziella'] = 102
autoresrjid['Carvalho, Frederico Antonio A'] = 113
autoresrjid['Carvalho, Frederico Antonio Azevedo De'] = 113
autoresrjid['Castilho, Marta Reis'] = 126
autoresrjid['Earp, Fabio Sa'] = 191
autoresrjid['Faria, Marina Dias De'] = 204
autoresrjid['Figueiredo, Klebler Fossati'] = 226
autoresrjid['Lima, Fernando Cerqueira'] = 317
autoresrjid['Siscu, Joao'] = 564
autoresrjid['Summa, Ricardo Figueiredo'] = 598
autoresrjid['Wanke, Peter Fernandes'] = 640

In [ ]:
# Criando a coluna de autor com o ID
dfrj['autor'] = dfrj['autor'].map(autoresrjid)

# Criando as colunas de coautores com ID
for t in range(1, max([len(dfrj['autores'].str.split('; ')[i]) for i in range(len(dfrj))])):
    string = 'coautor' + str(t)
    dfrj[string] = dfrj[string].map(autoresrjid).fillna('')

In [ ]:
# Verificando o DataFrame
dfrj

### Lista de número de artigos por autor/coautor

In [ ]:
# Lista de todos os ID's de autores
list(np.unique(list(autoresrjid.values())))

In [ ]:
# Dimensão dessa lista
len(list(np.unique(list(autoresrjid.values()))))

In [ ]:
# Criando um dicionário que associa o número de artigos publicados por autor ao seu ID
nartigosrj = {}
for i in list(np.unique(list(autoresrjid.values()))):
    k = i
    v = dfrj.loc[dfrj['autor'] == i].autor.count()
    for t in range(1, 9):
        string = 'coautor' + str(t)
        v = v + dfrj.loc[dfrj[string] == i].autor.count()
    nartigosrj[k] = v

In [ ]:
# Verificando o dicionário
nartigosrj

### Lista de edges

In [ ]:
# Criando uma lista dos edges a serem desenhados, associando o ID do autor com os respectivos ID's de seus coautores
edgesrj = []
for t in range(
        1,
    (max([len(dfrj['autores'].str.split('; ')[i])
          for i in range(len(dfrj))]))):
    for i in dfrj[dfrj['coautor' + str(t)] != ''][[
            'autor', 'coautor' + str(t)
    ]].itertuples(index=False, name=None):
        edgesrj.append(i)

In [ ]:
# Verificando as edges
edgesrj

### DataFrame de referência

In [ ]:
# Criando uma lista de referência com o ID de todos os autores
referenciarj = [i[t] for i in edgesrj for t in range(2)] + list(set(np.unique(list(
    autoresrjid.values()))) - set(np.unique([i[t] for i in edgesrj for t in range(2)])))

In [ ]:
# Criando um DataFrame de referência para identificar os ID's dos autores no grafo a ser criado adiante.
referenciarj = pd.DataFrame(referenciarj, columns=[
                            'ID']).drop_duplicates().reset_index(drop=True)

In [ ]:
# Adicionando a coluna do nome dos autores e seus respectivos número de artigos publicados na base de dados
autoresrjid = {y: x for x, y in autoresrjid.items()}
referenciarj['autores'] = referenciarj['ID'].map(autoresrjid)
referenciarj['nartigos'] = referenciarj['ID'].map(nartigosrj)

In [ ]:
# Criando a coluna com os ID's
referenciarj['ID'] = referenciarj.index

In [ ]:
# Verificando o DataFrame de referência
referenciarj

# Elaborando o grafo do IE-Unicamp

## Grafo da rede completa

In [ ]:
# Criando o grafo não direcionado e algumas propriedades dos vértices e edges
g = Graph(directed=False)
eprop = g.new_ep('int32_t')
for v in g.vertices():
    g.vp[v] = g.new_vp('int32_t')

# Adicionando os edges
g.add_edge_list(edges, eprops=eprop, hashed=True)

# Verificando o grafo
g

In [ ]:
# Adicionando os vértices dos autores que não possuem coautor (artigos de 1 autor)
g.add_vertex(len(np.unique(list(autoresid.values()))) - len(np.unique(edges)))

# Verificando o grafo
g

In [ ]:
# Configuração para o tamanho do vértice ser associada ao número de artigos publicados por autor
vsize = g.new_vp('int32_t')
vsize.a = ((np.array(referencia['nartigos'])) ** 20)
np.sqrt(max(vsize))

In [ ]:
# Configuração para as cores dos vértices serem associadas ao número de artigos publicados pelo autor, em um gradiente
vcolor = g.new_vp('double')
vcolor.a = referencia['nartigos']

In [ ]:
# Desenhando o grafo
graph_draw(g, output_size=(1800, 1800), vcmap=mpl.cm.YlGn,
           vertex_fill_color=vcolor, vertex_text=g.vertex_index, edge_pen_width=1.5)

## Elaborando o grafo dos componentes

### Maior componente

In [ ]:
# Extraindo o maior componente do grafo
largest = topology.label_largest_component(g)

In [ ]:
# Filtrando o grafo pelo maior componente
g.set_vertex_filter(largest)

In [ ]:
# Desenhando o maior componente
graph_draw(g, output_size=(800, 800), vertex_text=g.vertex_index)

### Outros componentes

In [ ]:
# Retirando o filtro utilizado anteriormente
# O objeto de grafo g retorna a seu tamanho anterior (completo)
g.set_vertex_filter(None)

In [ ]:
# Extraindo os componentes do grafo

# A função retorna dois objetos: um mapa de propriedades das vértices (comp) e um histograma (hist), este último utilizado para os gráficos
comp, hist = topology.label_components(g)

In [ ]:
# Extraindo um componente do grafo (no caso, o componente que contém o vértice 86)
comp1 = topology.label_out_component(g, g.vertex(86))

In [ ]:
# Filtrando o grafo pelo componente selecionado
g.set_vertex_filter(comp1)

In [ ]:
# Desenhando o componente selecionado
graph_draw(g, output_size=(1800, 1800), vertex_text=g.vertex_index)

### Elaborando os gráficos

In [ ]:
# Verificando o vetor do histograma
pd.Series(hist).value_counts()

Gráfico 1

Observações:
- O graph-tool cria os componentes arbitrariamente, associando eles com um número entre 0 a (N-1), onde N é o número total de componentes. Assim, o número do componente é pouco significativo. Por exemplo, o componente número 11 é o que tem mais vértices (43).
- Eixo x mostra o número de vértices
- Eixo y mostra o número de componentes conforme o número de vértices que eles possuem. Ou seja, são mais de 20 componentes com 1 vértice, enquanto existe apenas um único componente com 43 vértices.

In [ ]:
# Gráfico 1

fig, ax = plt.subplots(figsize=(8, 5))
sns.set_context('paper')
sns.set_style("white", {"xtick.major.size": 2, "ytick.major.size": 2})
sns.despine()
mpl.rcParams['font.family'] = 'sans-serif'

# Informando os dados para plotar
pd.Series(hist).value_counts().sort_index().plot(kind='bar',
                                                 subplots=True,
                                                 color='red',
                                                 ax=ax,
                                                 lw=1.5,
                                                 label='')

# Colocando o título
ax.set_title(label='Número de componentes e número de vértices por componente - Unicamp',
             fontsize=12)

# Ajustando o nome do eixo x
plt.ylabel('Número de componentes')
plt.xlabel('Número de vértices por componente')

# Ajustando os ticks dos eixos
ax.tick_params(axis='both', which='major', labelsize=10)
plt.xticks(rotation=45)

# Colocando a média, mediana e uma linha para indicar componentes únicos
plt.axhline(y=1,
            color='black',
            linestyle='-',
            alpha=0.7,
            zorder=3,
            label='Linha de componentes únicos')

# Colocando a legenda
plt.legend(loc='upper right', fontsize=12,
           frameon=True).get_frame().set_edgecolor('black')

# Implementando o tight_layout
plt.tight_layout()

# Plotando
plt.show()

# Elaborando o grafo do IE-URFJ

## Grafo da rede completa

In [ ]:
# Criando o grafo não direcionado e algumas propriedades dos vértices e edges
grj = Graph(directed=False)
eprop = grj.new_ep('int32_t')
for v in grj.vertices():
    grj.vp[v] = grj.new_vp('int32_t')

# Adicionando os edges
grj.add_edge_list(edgesrj, eprops=eprop, hashed=True)

# Verificando o grafo
grj

In [ ]:
# Adicionando os vértices dos autores que não possuem coautor (artigos de 1 autor)
grj.add_vertex(len(np.unique(list(autoresrjid.values()))) -
               len(np.unique(edgesrj)))

# Verificando o grafo
grj

In [ ]:
# Configuração para o tamanho do vértice ser associada ao número de artigos publicados por autor
vsize = grj.new_vp('int32_t')
vsize.a = ((np.array(referenciarj['nartigos'])) ** 1.7)
np.sqrt(max(vsize))

In [ ]:
# Configuração para as cores dos vértices serem associadas ao número de artigos publicados pelo autor, em um gradiente
vcolor = grj.new_vp('double')
vcolor.a = referenciarj['nartigos']

In [ ]:
# Desenhando o grafo
graph_draw(grj, output_size=(1800, 1800), vcmap=mpl.cm.YlGn,
           vertex_fill_color=vcolor, vertex_text=grj.vertex_index, edge_pen_width=1.5)

## Elaborando o grafo dos componentes

### Maior componente

In [ ]:
# Extraindo o maior componente do grafo
largest = topology.label_largest_component(grj)

In [ ]:
# Filtrando o grafo pelo maior componente
grj.set_vertex_filter(largest)

In [ ]:
# Desenhando o maior componente
graph_draw(grj, output_size=(800, 800), vertex_text=grj.vertex_index)

### Outros componentes (extraindo o componente do grupo do Excedente)

In [ ]:
# Retirando o filtro utilizado anteriormente
grj.set_vertex_filter(None)

In [ ]:
# Extraindo os componentes do grafo

# A função retorna dois objetos: um mapa de propriedades das vértices (comp) e um histograma (hist), este último utilizado para os gráficos
comp, hist = topology.label_components(grj)

In [ ]:
# Extraindo o componente relativo ao grupo do Excedente

# Buscando um ID de um autor pertencente ao grupo
referenciarj[referenciarj['autores'] == 'Serrano, Franklin']

In [ ]:
# Extraindo o componente que contém o vértice 39
comp1 = topology.label_out_component(grj, grj.vertex(39))

In [ ]:
# Filtrando o grafo pelo componente selecionado
grj.set_vertex_filter(comp1)

In [ ]:
# Desenhando o componente selecionado
graph_draw(grj, output_size=(1800, 1800), vertex_text=grj.vertex_index)

### Elaborando os gráficos

In [ ]:
# Verificando o vetor do histograma
pd.Series(hist).value_counts()

Gráfico 2

Observações:
- O graph-tool cria os componentes arbitrariamente, associando eles com um número entre 0 a (N-1), onde N é o número total de componentes. Assim, o número do componente é pouco significativo. Por exemplo, o componente número 11 é o que tem mais vértices (43).
- Eixo x mostra o número de vértices
- Eixo y mostra o número de componentes conforme o número de vértices que eles possuem. Ou seja, são mais de 50 componentes com 1 vértice, enquanto existe apenas um único componente com 43 vértices.

In [ ]:
# Gráfico 2

fig, ax = plt.subplots(figsize=(8, 5))
sns.set_context('paper')
sns.set_style("white", {"xtick.major.size": 12, "ytick.major.size": 12})
sns.despine()
mpl.rcParams['font.family'] = 'sans-serif'

# Informando os dados para plotar
pd.Series(hist).value_counts().sort_index().plot(kind='bar',
                                                 subplots=True,
                                                 color='red',
                                                 ax=ax,
                                                 lw=1.5,
                                                 label='')

# Colocando o título
ax.set_title(label='Número de componentes e número de vértices por componente - UFRJ',
             fontsize=12)

# Ajustando o nome do eixo x
plt.ylabel('Número de componentes')
plt.xlabel('Número de vértices por componente')
# Ajustando os ticks dos eixos
ax.tick_params(axis='both', which='major', labelsize=12)
plt.xticks(rotation=45)

# Colocando a média, mediana e uma linha para indicar componentes únicos
plt.axhline(y=1,
            color='black',
            linestyle='-',
            alpha=0.7,
            zorder=3,
            label='Linha de componentes únicos')

# Colocando a legenda
plt.legend(loc='upper right', fontsize=12,
           frameon=True).get_frame().set_edgecolor('black')

# Implementando o tight_layout
plt.tight_layout()

# Plotando
plt.show()